In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

In [3]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

In [4]:
simp = PlaintextCorpusReader(datapath+'/Corpora/wiki/simple_20200601/','simple_20200601_v2.txt',
                            word_tokenizer = WhitespaceTokenizer()
                            )

In [5]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

In [6]:
# Load n-gram dataframe produced in word2vec iteration

ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/ngram_eval.pkl')

In [7]:
ngram_eval

,ngram,freq,poisson,len,batch
0,"(Ving, Rhames)",20,-605.625590,2,1
1,"(Grădina, Zoologică)",20,-605.625590,2,1
2,"(Gharb-Chrarda-Beni, Hssen)",20,-605.625590,2,1
3,"(Karlovy, Vary)",20,-607.033377,2,1
4,"(waystations, shuku-eki)",20,-607.033377,2,1
...,...,...,...,...,...
149995,"(first, country, to)",45,-3071.582792,3,2
149996,"(from, A)",58,-3071.612088,2,-2
149997,"(1998, 8)",64,-3071.704678,2,6
149998,"(had, tested, positive)",55,-3071.728206,3,4


In [8]:
eval_count = 150000
min_freq = 20

In [9]:
ngram_eval.batch.value_counts()

 1     45301
 2     33312
 3     24686
 4     16668
 5     10517
 6      7502
 7      4240
 8      2702
 9      1905
 10     1051
-2       832
 11      450
-1       300
 12      247
 13      123
 14      107
 15       57
Name: batch, dtype: int64

In [10]:
# No. of simplex words to include in vocabulary (plus n-grams for each batch)
simplex_vocab_count = 300000

batch_count = max(ngram_eval.batch)
batch_count

15

In [11]:
import pickle
from gensim import corpora
from scipy import sparse
import itertools

from dist_cooccurrence import dist_cooccurrence_v3

In [12]:
#batch_count = 1

In [13]:
from nltk import FreqDist

fdist = FreqDist(word_gen(simp, sent_mark=''))

#stop = set( word for word, f in fdist.most_common(50))

In [14]:
fdist.N()

30471577

In [15]:
len(fdist)

1014614

In [16]:
# vocab = [w for w,f in filter(lambda x: x[1]>=20,fdist.items())]

In [17]:
simplex_vocab = set( word for word, f in fdist.most_common(simplex_vocab_count))
len(simplex_vocab)

300000

In [18]:
# Iterate over batches - save vocabs
for bb in range(batch_count):

    print('Dictionary for batch {}'.format(bb+1))
    
    this_batch = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    #  Append n-grams to simplex_vocab
    ng_list = [ '+'.join(ng) for ng in this_batch.ngram ]
    
    batch_vocab = simplex_vocab.copy()
    batch_vocab.update(ng_list)

    #  Pickle and save
    with open(datapath+'/Corpora/wiki/simple_20200601/simp_b{}vocab.pkl'.format(bb+1), 'wb') as pfile:
        pickle.dump(batch_vocab, pfile)

Dictionary for batch 1
Dictionary for batch 2
Dictionary for batch 3
Dictionary for batch 4
Dictionary for batch 5
Dictionary for batch 6
Dictionary for batch 7
Dictionary for batch 8
Dictionary for batch 9
Dictionary for batch 10
Dictionary for batch 11
Dictionary for batch 12
Dictionary for batch 13
Dictionary for batch 14
Dictionary for batch 15


In [19]:
# Dictionaries only


# Iterate over batches
for bb in range(batch_count):

    print('Dictionary for batch {}'.format(bb+1))

    #  Load pickled vocab set
    with open(datapath+'/Corpora/wiki/simple_20200601/simp_b{}vocab.pkl'.format(bb+1), 'rb') as pfile:
        batch_vocab = pickle.load(pfile)
        
    # Dictionary mapping
    batch_dict = corpora.Dictionary([['<unk>']])
    batch_dict.add_documents([list(batch_vocab)])
    
    # Pickle and save dictionary
    with open(datapath+'/Corpora/wiki/simple_20200601/Pickles/simp_b{}_dict.pkl'.format(bb+1), 'wb') as pfile:
        pickle.dump(batch_dict, pfile)

Dictionary for batch 1
Dictionary for batch 2
Dictionary for batch 3
Dictionary for batch 4
Dictionary for batch 5
Dictionary for batch 6
Dictionary for batch 7
Dictionary for batch 8
Dictionary for batch 9
Dictionary for batch 10
Dictionary for batch 11
Dictionary for batch 12
Dictionary for batch 13
Dictionary for batch 14
Dictionary for batch 15


In [31]:
%%time

# Iterate over batches
for bb in range(batch_count):

    print('Executing for batch {}'.format(bb+1))
    
    this_batch = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)

    #  Load pickled vocab set
    with open(datapath+'/Corpora/wiki/simple_20200601/simp_b{}vocab.pkl'.format(bb+1), 'rb') as pfile:
        batch_vocab = pickle.load(pfile)
        
    # Dictionary mapping
    batch_dict = corpora.Dictionary([['<unk>']])
    batch_dict.add_documents([list(batch_vocab)])
    
    # Pickle and save dictionary
    with open(datapath+'/Corpora/wiki/simple_20200601/Pickles/simp_b{}_dict.pkl'.format(bb+1), 'wb') as pfile:
        pickle.dump(batch_dict, pfile)
    
    
    # Distributed construction of matrix
    cooccurrence = dist_cooccurrence_v3(batch_dict, 
                                     sent_gen(simp, vocab=batch_vocab), 
                                     window_size = 20,
                                        mark=100000,
                                        rem_return = 50000,
                                    specs=[('popen', 6)])
    
    # Pickle and save
    with open(datapath+'/Corpora/wiki/simple_20200601/Pickles/simp_b{}_coo.pkl'.format(bb+1), 'wb') as pfile:
        pickle.dump(cooccurrence, pfile, protocol=4)
        
    print('Pickled')
    print('****************************','\n')
    
    del cooccurrence
    

Executing for batch 1
 Processed: 100000
 Processed: 200000
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
 Processed: 300000
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
 Processed: 400000
 Processed: 500000
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
 Processed: 600000
Obtained response from channel <Channel id=1 open>
 Processed: 700000
 Processed: 800000
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
 Processed: 900000
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id

Started 11.58

Sentences:
16500000 - 16600000